In [1]:
# Install dependencies (run once)
!pip install telethon python-dotenv aiofiles


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import json
import logging
from datetime import datetime
from telethon.sync import TelegramClient
from telethon.tl.types import MessageMediaPhoto
from dotenv import load_dotenv
from pathlib import Path


In [3]:
# Load environment
load_dotenv()
api_id = int(os.getenv("TELEGRAM_API_ID"))
api_hash = os.getenv("TELEGRAM_API_HASH")

In [4]:

# Setup logging
logging.basicConfig(filename="scraping.log", level=logging.INFO)

# Define target channels
channels = {
    "lobelia4cosmetics": "https://t.me/lobelia4cosmetics",
    "tikvahpharma": "https://t.me/tikvahpharma"
}

In [8]:
# Define base directory
BASE_DIR = Path("../data/raw/telegram_messages")
IMAGE_DIR = Path("../data/raw/images")
BASE_DIR.mkdir(parents=True, exist_ok=True)
IMAGE_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
import asyncio
from telethon.sync import TelegramClient
from telethon.tl.types import MessageMediaPhoto

async def scrape_telegram():
    async with TelegramClient("session_scraper", api_id, api_hash) as client:
        for channel_name, url in channels.items():
            logging.info(f"Scraping {channel_name}")
            try:
                today = datetime.now().strftime("%Y-%m-%d")
                save_path = BASE_DIR / today / f"{channel_name}.json"
                save_path.parent.mkdir(parents=True, exist_ok=True)

                messages_data = []
                async for message in client.iter_messages(url, limit=100):
                    msg = {
                        "id": message.id,
                        "date": str(message.date),
                        "sender": str(message.sender_id),
                        "text": message.text,
                    }

                    # Save image if available
                    if message.media and isinstance(message.media, MessageMediaPhoto):
                        img_path = IMAGE_DIR / f"{channel_name}_{message.id}.jpg"
                        await client.download_media(message, file=img_path)
                        msg["image_path"] = str(img_path)

                    messages_data.append(msg)

                with open(save_path, "w", encoding="utf-8") as f:
                    json.dump(messages_data, f, indent=2, ensure_ascii=False)

                logging.info(f"Saved {len(messages_data)} messages for {channel_name}")
            except Exception as e:
                logging.error(f"Failed to scrape {channel_name}: {e}")

# Run the async function
await scrape_telegram()


Signed in successfully as teme; remember to not break the ToS or you will risk an account ban!
